## Concrete steps
### Imports

In [50]:
#%% imports
import requests
from lxml import html, etree
import pandas as pd
from urllib.parse import urljoin
import numpy as np
import pickle
import re

#%%
def get(url):
	"""Uses HTTP-GET to request a webresource and converts the result into html-object."""
	result = sessionRequests.get(url
		,headers=dict(referer = url))
	return html.fromstring(result.content)


### Crawl the [main hpi courses website]("https://hpi.de/studium/lehrveranstaltungen") to obtain a list of courselists (eg. Bachelor, Master courses, etc.) 

In [3]:
#%%
sessionRequests = requests.session()

baseURL = "https://hpi.de/studium/lehrveranstaltungen"

In [51]:
def getLVOverwievs(url) -> dict:
	"""crawls the main hpi courses website to obtain a list of courselists (eg. Bachelor, Master courses, etc.) """
	lvSuperList = get(url)
	lvOverviews = lvSuperList.xpath("//h1[contains(text(),'Lehrveranstaltungen')]//following-sibling::ul//a")
	return {x.text:x.get("href") for x in lvOverviews}

lvOverviews = getLVOverwievs(baseURL)

#expand to full path
lvOverviews = list(map(lambda x:urljoin(baseURL,x),lvOverviews.values()))
lvOverviews

['https://hpi.de/studium/lehrveranstaltungen/it-systems-engineering-ba.html',
 'https://hpi.de/studium/lehrveranstaltungen/it-systems-engineering-ma.html',
 'https://hpi.de/studium/lehrveranstaltungen/digital-health.html',
 'https://hpi.de/studium/lehrveranstaltungen/data-engineering.html',
 'https://hpi.de/studium/lehrveranstaltungen/soft-skills.html',
 'https://hpi.de/entrepreneurship/veranstaltungen/it-entrepreneurship-vorlesung.html',
 'https://hpi.de/studium/lehrveranstaltungen/design-thinking.html']

### Crawl each table-page with courses and returns a dict with coursename and URL to coursepage.

In [52]:
def extractDictOfCourses(url):
	"""Crawls a table-page with courses and returns a dict coursename and URL to coursepage """
	courseListPage = get(url)
	return {item.text.strip(): urljoin(url,item.get("href")) for item in
		 courseListPage.xpath("//div[@id = 'content']//table//tr//td[1]//a")}

#list of Courses
listOfCourses = list (map(extractDictOfCourses,lvOverviews))
listOfCourses

[{'3D-Computergrafik I': 'https://hpi.de/studium/lehrveranstaltungen/it-systems-engineering-ba/lehrveranstaltung/course/0/wintersemester-20182019-3d-computergrafik-i.html',
  'Algorithmic Problem Solving': 'https://hpi.de/studium/lehrveranstaltungen/it-systems-engineering-ba/lehrveranstaltung/course/0/wintersemester-20182019-algorithmic-problem-solving.html',
  'Betriebssysteme I': 'https://hpi.de/studium/lehrveranstaltungen/it-systems-engineering-ba/lehrveranstaltung/course/0/wintersemester-20182019-betriebssysteme-i.html',
  'Building Interactive Devices': 'https://hpi.de/studium/lehrveranstaltungen/it-systems-engineering-ba/lehrveranstaltung/course/0/wintersemester-20182019-building-interactive-devices.html',
  'D-School Advanced Track': 'https://hpi.de/studium/lehrveranstaltungen/it-systems-engineering-ba/lehrveranstaltung/course/0/wintersemester-20182019-d-school-advanced-track.html',
  'D-School Basic Track': 'https://hpi.de/studium/lehrveranstaltungen/it-systems-engineering-ba/l

### Combine the list of dicts to one big list, which only contains URLs.

In [53]:
def combineLinkList(linkList):
	"""combines the list of dicts to one big list, which only contains URLs"""
	return [courseLink for subject in 
		linkList for courseLink in subject.values()]

urlsToCrawl = combineLinkList(listOfCourses)

### Downloads the individual course websites.

In [7]:
#%%
#import time
#def downloadCourseSites(urls,delay = 0.1):
#	"""downloads the individual course websites """
#	result = []
#	for item in urls:
#		result.append(get(item))
#		print(f"got {item.split('/')[-1]}")
#		time.sleep(delay)
#	return result

#courseWebsites = downloadCourseSites (urlsToCrawl)
#%% save/load crawled websites with pickle

# courseWebsitesString = [etree.tostring(x) for x in courseWebsites]

# with open("websites.pkl","wb") as file:
# 	pickle.dump(courseWebsitesString, file)

### or use local copy instead

In [54]:
#%%
"""uncomment this to load the locally saved websites"""
with open("websites.pkl","rb") as file:
	cws = pickle.load(file)

courseWebsites = [html.fromstring(x) for x in cws]

### Extract data from course website
5. Use XPATH to extract information from the course websites. Store it in a dict.
6. If coursewebsite links to a coursesite from the chair, download the chair site as well.
7. Finds all Links, on the websites that destination ends with the given fileextension (eg. '.pdf', '.pptx').


In [55]:
#%%
def findAllFiles(secondPage, secondURL, fileExtension):
    """Finds all Links, on given website at given url that destination ends with
    the given fileextension."""
    assert secondPage, "Second page must not be 'None'."
    return [urljoin(secondURL,x) for x in 
        secondPage.xpath(f"//a[contains(@href,'{fileExtension}')]/@href")]

#%%
"""lets create a list of dicts where each dict holds infos about one course"""
allCourses = []
for site,url in zip(courseWebsites, urlsToCrawl):
    course = {}
    #get the main heading of the coursesite which should contain the coursename and semester
    fullTitle = site.xpath("//h1[1]/text()")[0]
    #use this regex to split coursename and semester
    matches = re.search(r"(.*)\((.*)\)",fullTitle)

    if matches:
        course["Name"] = matches[1]
        course["Semester"] = matches[2]
    else:
        #if regex did not match anything, just use the whole headline as coursetitle and 
        #assume theres no semester given
        course["Name"] = fullTitle
    course["URL"] = url
    #find the link after '...Website', which should be the link to the 2nd course page
    secondURL = site.xpath("//div/text()[contains(.,'Website')]/following-sibling::i/a/@href")
    secondURL = urljoin(url, secondURL[0]) if secondURL else None
    secondPage = None
    if secondURL:
        course["secondURL"] = secondURL
        secondPage = get(secondURL)
        course["files"] = findAllFiles(secondPage, secondURL, ".pdf")
    
    #Margaux Anrechenbarkeit
    anItSe = site.xpath("//div[contains(text(),'IT-Systems Engineering MA')]//following-sibling::ul//li//text()")
    str_anItSe = ", ".join(anItSe)
    course["Anrechenbarkeit Master ItSe"] = str_anItSe
    
    anDE = site.xpath("//div[contains(text(),'Data Engineering MA')]//following-sibling::ul//li//text()")
    str_anDE = ", ".join(anDE)
    course["Anrechenbarkeit Master DE"] = str_anDE
    
    anDH = site.xpath("//div[contains(text(),'Digital Health MA')]//following-sibling::ul//li//text()")
    str_anDH = ", ".join(anDH)
    course["Anrechenbarkeit Master DH"] = str_anDH
    
    anB = site.xpath("//div[contains(text(),'IT-Systems Engineering BA')]//following-sibling::ul//li//text()")
    str_anB = ", ".join(anB)
    course["Anrechenbarkeit Bachelor ItSe"] = str_anB  
    
    #extract all basic infos from a list on the course page
    for item in site.xpath("//h2[contains(text(),'Allgemeine Information')]/following-sibling::ul[1]//li//text()"):
        if len(item.split(":")) == 2:
            key,val = item.split(":")
            course[key] = val.strip()
    allCourses.append(course)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  """


### Create Pandas dataframe

In [56]:
#%%
"""Create a DataFrame from the dicts"""
df = pd.DataFrame(columns=["Name"
    #,"Website"
    ,"Belegungsart"
    ,"Benotet"
    ,"ECTS"
    ,"Einschreibefrist"
    ,"Lehrform"
    ,"Lehrsprache"
    ,"Semesterwochenstunden"
    ,"Anrechenbarkeit Bachelor ItSe"
    ,"Anrechenbarkeit Master ItSe"
    ,"Anrechenbarkeit Master DE"
    ,"Anrechenbarkeit Master DH"
    ,"URL"
    ,"files"
    ,"secondURL"
    ])
df = df.append(allCourses,ignore_index = True)
df[df.Name.str.contains("Preparation")]

,Anrechenbarkeit Bachelor ItSe,Anrechenbarkeit Master DE,Anrechenbarkeit Master DH,Anrechenbarkeit Master ItSe,Belegungsart,Benotet,ECTS,Einschreibefrist,Lehrform,Lehrsprache,Maximale Teilnehmerzahl,Name,Semester,Semesterwochenstunden,URL,files,secondURL
46,,"DATA-Konzepte und Methoden, DATA-Techniken und...","APAD-Concepts and Methods, APAD-Technologies a...","OSIS-Konzepte und Methoden, OSIS-Spezialisieru...",Wahlpflichtmodul,Ja,6,26.10.2018,Projektseminar,Deutsch,NaN,Data Preparation for Science,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[https://hpi.de/fileadmin/user_upload/fachgebi...,https://hpi.de/naumann/teaching/teaching/ws-18...
96,,"DATA-Konzepte und Methoden, DATA-Techniken und...","APAD-Concepts and Methods, APAD-Technologies a...","OSIS-Konzepte und Methoden, OSIS-Spezialisieru...",Wahlpflichtmodul,Ja,6,26.10.2018,Projektseminar,Deutsch,NaN,Data Preparation for Science,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/dig...,[https://hpi.de/fileadmin/user_upload/fachgebi...,https://hpi.de/naumann/teaching/teaching/ws-18...
129,,"DATA-Konzepte und Methoden, DATA-Techniken und...","APAD-Concepts and Methods, APAD-Technologies a...","OSIS-Konzepte und Methoden, OSIS-Spezialisieru...",Wahlpflichtmodul,Ja,6,26.10.2018,Projektseminar,Deutsch,NaN,Data Preparation for Science,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/dat...,[https://hpi.de/fileadmin/user_upload/fachgebi...,https://hpi.de/naumann/teaching/teaching/ws-18...


### Use pandas built in methods to remove duplicates


In [64]:
#remove duplicates, but ignore files and URL column
duplicatesRemoved = df.drop_duplicates(df.columns.difference(["files","URL"]))
duplicatesRemoved
dup2 = df.drop_duplicates(subset =["Name"])

In [67]:
print(f"with duplicates: {len(df)}")
print(f"with duplicates removed: {len(duplicatesRemoved)}")
print(len(dup2))

merged = duplicatesRemoved.columns.difference(["files","URL"]).merge(dup2, indicator=True, how='outer')
merged[merged['_merge'] == 'right_only']

with duplicates: 164
with duplicates removed: 99
96


TypeError: unhashable type: 'list'

In [59]:
duplicatesRemoved[duplicatesRemoved.Name.str.contains("D-School Basic Track")]

,Anrechenbarkeit Bachelor ItSe,Anrechenbarkeit Master DE,Anrechenbarkeit Master DH,Anrechenbarkeit Master ItSe,Belegungsart,Benotet,ECTS,Einschreibefrist,Lehrform,Lehrsprache,Maximale Teilnehmerzahl,Name,Semester,Semesterwochenstunden,URL,files,secondURL
5,Design Thinking,Design Thinking Basics,Design Thinking Basic,Design Thinking Basic,Wahlpflichtmodul,Ja,9,31.01.2019,Projekt / Seminar,NaN,80,D-School Basic Track,Wintersemester 2018/2019,6,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
162,,,,Design Thinking Advanced,,Ja,9,31. Juli 2018 für das Wintersemester 2018/2019,Projekt / Seminar,NaN,80,D-School Basic Track,Sommersemester 2018,6,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN


In [60]:
list(duplicatesRemoved[duplicatesRemoved.Name.str.contains("Preparation")].files)

[['https://hpi.de/fileadmin/user_upload/fachgebiete/naumann/lehre/WS2018/Data_preparation_for_science/01_Intro_DQ_DataPrep_Felix.pdf',
  'https://hpi.de/fileadmin/user_upload/fachgebiete/naumann/lehre/WS2018/Data_preparation_for_science/Task_proposals.pdf',
  'https://hpi.de/fileadmin/user_upload/fachgebiete/naumann/lehre/WS2018/Data_preparation_for_science/03_task_announcement.pdf']]

In [61]:
duplicatesRemoved[(duplicatesRemoved["Lehrform"].str.contains("Vorlesung", na=False))
                      & (duplicatesRemoved.ECTS.astype(float) == 6) & duplicatesRemoved["Anrechenbarkeit Master ItSe"]]

,Anrechenbarkeit Bachelor ItSe,Anrechenbarkeit Master DE,Anrechenbarkeit Master DH,Anrechenbarkeit Master ItSe,Belegungsart,Benotet,ECTS,Einschreibefrist,Lehrform,Lehrsprache,Maximale Teilnehmerzahl,Name,Semester,Semesterwochenstunden,URL,files,secondURL
36,,"DATA-Konzepte und Methoden, DATA-Techniken und...",SCAD-Specialization,"ISAE-Konzepte und Methoden, ISAE-Techniken und...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Englisch,10,Advanced Probability Theory,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[],https://hpi.de/friedrich/teaching/ws18/probabi...
37,,"SCAL-Konzepte und Methode, SCAL-Techniken und ...","SCAD-Concepts and Methods, SCAD-Technologies a...","ITSE-Analyse, ITSE-Entwurf, ITSE-Konstruktion,...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,NaN,NaN,Algorithmix,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[],https://hpi.de//friedrich/teaching/ws18/algori...
38,,"SCAL-Konzepte und Methode, SCAL-Techniken und ...",SCAD-Specialization,"ISAE-Konzepte und Methoden, ISAE-Techniken und...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Deutsch,30,Approximation Algorithms,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[],https://hpi.de//friedrich/teaching/ws18/approx...
41,,"DATA-Konzepte und Methoden, DATA-Techniken und...",SCAD-Specialization,"ISAE-Konzepte und Methoden, ISAE-Spezialisieru...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Deutsch,30,Computational Geometry,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[],https://hpi.de/friedrich/teaching/ws18/algorit...
45,,"CODS-Konzepte und Methoden, CODS-Techniken und...","SCAD-Concepts and Methods, SCAD-Technologies a...","ITSE-Analyse, ITSE-Entwurf, ITSE-Konstruktion,...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Englisch,NaN,Data Management for Digital Health,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[https://hpi.de/fileadmin/user_upload/fachgebi...,https://hpi.de/digital-health-center/teaching/...
50,,Datenmanagement,"APAD-Concepts and Methods, APAD-Technologies a...","ITSE-Analyse, ITSE-Entwurf, ITSE-Konstruktion,...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Deutsch,NaN,Distributed Data Management,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[https://hpi.de/fileadmin/user_upload/fachgebi...,https://hpi.de/naumann/teaching/teaching/ws-18...
52,,,"SCAD-Concepts and Methods, SCAD-Technologies a...","ITSE-Analyse, ITSE-Entwurf, ITSE-Konstruktion,...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung / Projekt,Deutsch,30,Embedded Operating Systems for Internet of Thi...,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[https://www.dcl.hpi.uni-potsdam.de/teaching/E...,https://www.dcl.hpi.uni-potsdam.de/teaching/Em...
66,,,,"ITSE-Analyse, ITSE-Entwurf, ITSE-Konstruktion,...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung,Deutsch,NaN,Methods of Cloud Computing,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[https://www.dcl.hpi.uni-potsdam.de/teaching/m...,https://www.dcl.hpi.uni-potsdam.de/teaching/mo...
74,,,"APAD-Concepts and Methods, APAD-Technologies a...","ITSE-Analyse, ITSE-Entwurf, ITSE-Konstruktion,...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Deutsch,NaN,Safety-Critical Systems: From Predictable Syst...,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
79,,,,"OSIS-Konzepte und Methoden, OSIS-Spezialisieru...",Wahlpflichtmodul,Ja,6,26.10.2018,Vorlesung / Projekt,Deutsch,20,"Software Testen, Analysieren und Verifizieren",Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN


In [49]:
duplicatesRemoved[(duplicatesRemoved.Belegungsart == "Pflichtmodul")]

,Anrechenbarkeit Bachelor ItSe,Anrechenbarkeit Master DE,Anrechenbarkeit Master DH,Anrechenbarkeit Master ItSe,Belegungsart,Benotet,ECTS,Einschreibefrist,Lehrform,Lehrsprache,Maximale Teilnehmerzahl,Name,Semester,Semesterwochenstunden,URL,files,secondURL
2,Betriebssysteme,,,,Pflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Deutsch,NaN,Betriebssysteme I,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,[https://www.dcl.hpi.uni-potsdam.de/teaching/B...,https://www.dcl.hpi.uni-potsdam.de/teaching/BS/
9,Programmiertechnik I,,,,Pflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Deutsch,NaN,Einführung in die Programmiertechnik I,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
13,Digitale Systeme,,,,Pflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Deutsch,NaN,Grundlagen digitaler Systeme,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
17,Mathematik I,,,,Pflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Deutsch,NaN,Mathematik I - Diskrete Strukturen und Logik,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
18,Modellierung I,,,,Pflichtmodul,Ja,6,26.10.2018,Vorlesung / Übung,Deutsch,NaN,Modellierung I,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
22,,,,,Pflichtmodul,Ja,6,26.10.2018,Blockseminar,Deutsch,20,Projektentwicklung und - management: Prozess,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
23,Projektentwicklung und -management,,,,Pflichtmodul,Ja,6,26.10.2018,Blockseminar,NaN,20,Projektentwicklung und -management,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
24,Projektentwicklung und -management,,,,Pflichtmodul,Ja,6,,Blockseminar,Deutsch,20,Projektentwicklung und- Management: Teammanage...,Wintersemester 2018/2019,4,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
25,Rechtliche Grundlagen,,,,Pflichtmodul,Ja,3,26.10.2018,Vorlesung,Deutsch,NaN,Recht für Ingenieure I,Wintersemester 2018/2019,2,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN
26,Rechtliche Grundlagen,,,,Pflichtmodul,Ja,3,26.10.2018,Vorlesung,NaN,NaN,Recht für Ingenieure II,Wintersemester 2018/2019,2,https://hpi.de/studium/lehrveranstaltungen/it-...,NaN,NaN


### Save the result as .csv file.


In [ ]:
duplicatesRemoved.to_csv("output.csv")

### continue with OpenRefine